<a href="https://colab.research.google.com/github/beta996/Twitter-Sentiment-Analysis/blob/main/MLproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

/bin/bash: line 0: cd: too many arguments
